In [ ]:
import time
import pyvisa

# === CONFIGURATION ===
RESOURCE          = 'USB0::0x1313::0x80C8::M00811426::INSTR'  # ← your DC2200 VISA address
INITIAL_COUNTDOWN = 20    # seconds before first pulse
ON_TIME           = 10    # seconds LED stays ON
OFF_TIME          = 10    # seconds LED stays OFF
CURRENTS_MA       = list(range(0, 101, 20))  # [0, 20, 40, 60, 80, 100] mA

def countdown(t):
    for remaining in range(t, 0, -1):
        print(f'Starting in {remaining:2d} s…', end='\r')
        time.sleep(1)
    print(' ' * 30, end='\r')

def main():
    rm   = pyvisa.ResourceManager()
    inst = rm.open_resource(RESOURCE)
    inst.timeout = 5000  # 5 s

    # Reset & configure for constant-current
    inst.write('*RST')
    inst.write('SOURce1:MODe CC')                              # CC mode
    inst.write('SOURce1:CURRent:LIMit:AMPLitude 0.1')           # 100 mA limit

    print(f'Connected to {inst.query("*IDN?").strip()}')
    countdown(INITIAL_COUNTDOWN)

    for idx, I_mA in enumerate(CURRENTS_MA, start=1):
        I_A = I_mA / 1000.0
        print(f'\nPulse {idx}/{len(CURRENTS_MA)} — setting {I_mA} mA')
        inst.write(f'SOURce1:CURRent:LEVel:AMPLitude {I_A}')

        # ON phase
        try:
            inst.write('OUTPut ON')
            print(f'  ▶ LED ON at {I_mA} mA for {ON_TIME} s')
            for sec in range(ON_TIME, 0, -1):
                print(f'    {sec:2d}s remaining…', end='\r')
                time.sleep(1)
            print(' ' * 30, end='\r')

            # Measurement
            meas_I = float(inst.query('MEASure:CURRent:DC?')) * 1000
            meas_V = float(inst.query('MEASure:VOLTage:DC?'))
            print(f'  ☑ Measured: {meas_I:.1f} mA | {meas_V:.2f} V')

        except KeyboardInterrupt:
            print('\n⚠️  Interrupted during ON phase!')
        finally:
            inst.write('OUTPut OFF')
            print('  ▶ LED OFF for safety.')

        # OFF phase
        try:
            print(f'  ⏸ Cooling for {OFF_TIME} s…')
            for sec in range(OFF_TIME, 0, -1):
                print(f'    {sec:2d}s remaining…', end='\r')
                time.sleep(1)
            print(' ' * 30, end='\r')
        except KeyboardInterrupt:
            print('\n⚠️  Interrupted during OFF phase!')

        # Continue?
        cont = input('Perform next pulse? (yes/no): ').strip().lower()
        if cont not in ('yes', 'y'):
            print('Exiting pulse sequence.')
            break

    print('Done!')

if __name__ == '__main__':
    main()


In [ ]:
import usb.core
import usb.util

def list_usb_devices():
    devices = usb.core.find(find_all=True)
    for device in devices:
        print(f"ID {hex(device.idVendor)}:{hex(device.idProduct)} - Bus {device.bus} Address {device.address}")

if __name__ == "__main__":
    list_usb_devices()

In [ ]:
pip install pyusb

In [ ]:
import pyvisa
rm = pyvisa.ResourceManager()
print(rm.list_resources())

In [ ]:
pip install pyvisa

## trial 3

In [1]:
import time
import pyvisa

# === CONFIGURATION ===
RESOURCE          = 'USB0::0x1313::0x80C8::M00811426::INSTR'
INITIAL_COUNTDOWN = 5     # seconds before first pulse
ON_TIME           = 5     # seconds LED stays ON
OFF_TIME          = 5     # seconds LED stays OFF
CURRENTS_MA       = list(range(20, 101, 20))  # [20, 40, 60, 80, 100] mA

def countdown(sec):
    for t in range(sec, 0, -1):
        print(f'Starting in {t:2d}s…', end='\r')
        time.sleep(1)
    print(' ' * 30, end='\r')

def run_pulses(inst):
    """Run one full on/off sweep through all current steps."""
    for idx, I_mA in enumerate(CURRENTS_MA, start=1):
        I_A = I_mA / 1000.0
        print(f'\nPulse {idx}/{len(CURRENTS_MA)} → {I_mA} mA')

        # 1) Program the forward current in CC mode
        inst.write(f'SOURce1:CCURRent:LEVel:AMPLitude {I_A}')   # CC level :contentReference[oaicite:12]{index=12}

        # 2) Turn LED on
        inst.write('OUTPut1:STATe ON')                         # Output ON :contentReference[oaicite:13]{index=13}
        print(f'  ▶ ON for {ON_TIME}s')
        for t in range(ON_TIME, 0, -1):
            print(f'    {t:2d}s…', end='\r')
            time.sleep(1)
        print(' ' * 30, end='\r')

        # 3) Measure & report
        meas_I = float(inst.query('MEASure:CURRent:DC?')) * 1000
        meas_V = float(inst.query('MEASure:VOLTage:DC?'))
        print(f'  ☑ Measured: {meas_I:.1f} mA | {meas_V:.2f} V')

        # 4) Turn LED off
        inst.write('OUTPut1:STATe OFF')                        # Output OFF :contentReference[oaicite:14]{index=14}
        print(f'  ▶ OFF for {OFF_TIME}s')
        for t in range(OFF_TIME, 0, -1):
            print(f'    {t:2d}s…', end='\r')
            time.sleep(1)
        print(' ' * 30, end='\r')

def main():
    rm   = pyvisa.ResourceManager()
    inst = rm.open_resource(RESOURCE)
    inst.timeout = 5000  # 5 s timeout

    # --- INITIAL SETUP ---
    inst.write('*RST')
    inst.write('SOURce1:MODe CC')                              # Constant-Current mode
    inst.write('SOURce1:CURRent:LIMIt:AMPLitude 0.1')           # 100 mA limit :contentReference[oaicite:15]{index=15}
    print(f'Connected: {inst.query("*IDN?").strip()}')

    while True:
        countdown(INITIAL_COUNTDOWN)

        try:
            run_pulses(inst)
        except KeyboardInterrupt:
            # Immediately turn LED off on Ctrl+C
            print('\n⚠️  Interrupted! Turning LED off…')
            inst.write('OUTPut1:STATe OFF')                    # Safety OFF :contentReference[oaicite:16]{index=16}

        # Ask only once after completing or interrupting the full sweep
        ans = input('\nRun another full pulse sequence? (y/n): ').strip().lower()
        if ans not in ('y', 'yes'):
            print('Exiting. Goodbye!')
            break

if __name__ == '__main__':
    main()


Connected: Thorlabs,DC2200,M00811426,1.4.0
                              
Pulse 1/5 → 20 mA
  ▶ ON for 5s
  ☑ Measured: 100.0 mA | 2.83 V
  ▶ OFF for 5s
                              
Pulse 2/5 → 40 mA
  ▶ ON for 5s
  ☑ Measured: 100.0 mA | 2.83 V
  ▶ OFF for 5s
                              
Pulse 3/5 → 60 mA
  ▶ ON for 5s
  ☑ Measured: 100.0 mA | 2.83 V
  ▶ OFF for 5s
                              
Pulse 4/5 → 80 mA
  ▶ ON for 5s
  ☑ Measured: 100.0 mA | 2.83 V
  ▶ OFF for 5s
                              
Pulse 5/5 → 100 mA
  ▶ ON for 5s
  ☑ Measured: 100.0 mA | 2.83 V
  ▶ OFF for 5s
     1s…
⚠️  Interrupted! Turning LED off…
Exiting. Goodbye!


## LED 2 ctrl trial 5

In [4]:
import time
import pyvisa

# === CONFIGURATION ===
RESOURCE          = 'USB0::0x1313::0x80C8::M00811426::INSTR'
INITIAL_COUNTDOWN = 5     # seconds before first pulse
ON_TIME           = 5     # seconds LED stays ON
OFF_TIME          = 5     # seconds LED stays OFF
CURRENTS_MA       = list(range(20, 101, 20))  # [20, 40, 60, 80, 100] mA

def countdown(sec):
    for t in range(sec, 0, -1):
        print(f'Starting in {t:2d}s…', end='\r')
        time.sleep(1)
    print(' ' * 30, end='\r')

def run_sequence(inst):
    for idx, I_mA in enumerate(CURRENTS_MA, start=1):
        I_A = I_mA / 1000.0
        print(f'\nPulse {idx}/{len(CURRENTS_MA)} → {I_mA} mA')

        # 1) Program the forward current in CC mode on the *selected* terminal
        inst.write(f'SOURce1:CCURRent:CURRent:LEVel:AMPLitude {I_A}')  # correct CC path :contentReference[oaicite:10]{index=10}

        # 2) Turn LED on
        inst.write('OUTPut1:STATe ON')
        print(f'  ▶ ON for {ON_TIME}s')
        for t in range(ON_TIME, 0, -1):
            print(f'    {t:2d}s…', end='\r')
            time.sleep(1)
        print(' ' * 30, end='\r')

        # 3) Measure & report
        meas_I = float(inst.query('MEASure:CURRent:DC?')) * 1000     # measures on active terminal :contentReference[oaicite:11]{index=11}
        meas_V = float(inst.query('MEASure:VOLTage:DC?'))
        print(f'  ☑ Measured: {meas_I:.1f} mA | {meas_V:.2f} V')

        # 4) Turn LED off
        inst.write('OUTPut1:STATe OFF')
        print(f'  ▶ OFF for {OFF_TIME}s')
        for t in range(OFF_TIME, 0, -1):
            print(f'    {t:2d}s…', end='\r')
            time.sleep(1)
        print(' ' * 30, end='\r')

def main():
    rm   = pyvisa.ResourceManager()
    inst = rm.open_resource(RESOURCE)
    inst.timeout = 5000  # ms

    # --- INITIAL SETUP ---
    inst.write('*RST')
    inst.write('OUTPut1:TERMinal 2')                             # select LED2 terminal :contentReference[oaicite:12]{index=12}
    inst.write('SOURce1:MODe CC')                                # Constant-Current mode
    inst.write('SOURce1:CURRent:LIMIt:AMPLitude 0.1')            # 100 mA safety limit
    print(f'Connected: {inst.query("*IDN?").strip()}')

    while True:
        countdown(INITIAL_COUNTDOWN)
        try:
            run_sequence(inst)
        except KeyboardInterrupt:
            # Immediate safety shutdown on Ctrl+C
            print('\n⚠️  Interrupted! Turning LED off…')
            inst.write('OUTPut1:STATe OFF')

        # One single prompt after the full sweep (or an interrupt)
        ans = input('\nRun another full pulse sequence? (y/n): ').strip().lower()
        if ans not in ('y','yes'):
            print('Exiting. Goodbye!')
            break

if __name__ == '__main__':
    main()


Connected: Thorlabs,DC2200,M00811426,1.4.0
                              
Pulse 1/5 → 20 mA
  ▶ ON for 5s
  ☑ Measured: 100.1 mA | 2.83 V
  ▶ OFF for 5s
                              
Pulse 2/5 → 40 mA
  ▶ ON for 5s
  ☑ Measured: 100.0 mA | 2.83 V
  ▶ OFF for 5s
                              
Pulse 3/5 → 60 mA
  ▶ ON for 5s
  ☑ Measured: 100.0 mA | 2.84 V
  ▶ OFF for 5s
     4s…
⚠️  Interrupted! Turning LED off…
Exiting. Goodbye!


## pulse code command trial 6

In [ ]:
import time
import pyvisa

# === CONFIGURATION ===
RESOURCE          = 'USB0::0x1313::0x80C8::M00811426::INSTR'
INITIAL_COUNTDOWN = 5     # s before first sequence
ON_TIME           = 5     # s pulse ON
OFF_TIME          = 5     # s pulse OFF
CURRENTS_MA       = list(range(20, 201, 20))  # [20,40,…,200] mA
MAX_LIMIT_MA      = 200   # new user current limit (mA)


def countdown(sec):
    for t in range(sec, 0, -1):
        print(f'Starting in {t:2d}s…', end='\r')
        time.sleep(1)
    print(' ' * 40, end='\r')

def run_sequence(inst):
    for idx, I_mA in enumerate(CURRENTS_MA, start=1):
        pct = (I_mA / MAX_LIMIT_MA) * 100
        print(f'\nPulse {idx}/{len(CURRENTS_MA)} → {I_mA} mA ({pct:.0f}% of limit)')

        # Configure Pulse Mode
        inst.write('SOURce1:MODe PULS')                               
        inst.write(f'SOURce1:CURRent:LIMIt:AMPLitude {MAX_LIMIT_MA/1000}')  
        inst.write(f'SOURce1:PULSe:BRIGhtness:LEVel:AMPLitude {pct}')    # brightness % :contentReference[oaicite:12]{index=12}
        inst.write(f'SOURce1:PULSe:ONTime {ON_TIME}')                    # ON time :contentReference[oaicite:13]{index=13}
        inst.write(f'SOURce1:PULSe:OFFTime {OFF_TIME}')                  # OFF time :contentReference[oaicite:14]{index=14}
        inst.write('SOURce1:PULSe:COUNt 1')                             

        # Fire one pulse
        inst.write('OUTPut1:STATe ON')
        # wait exactly 1 s, then take reading
        time.sleep(1)
        meas_I1 = float(inst.query('MEASure:CURRent:DC?')) * 1000       # → mA :contentReference[oaicite:15]{index=15}
        meas_V1 = float(inst.query('MEASure:VOLTage:DC?'))              
        print(f'  ☑ [1 s] {meas_I1:.1f} mA | {meas_V1:.2f} V')

        # finish the rest of the ON period
        for t in range(ON_TIME - 1, 0, -1):
            print(f'    {t:2d}s remaining ON…', end='\r')
            time.sleep(1)
        print(' ' * 40, end='\r')

        # Turn OFF and query status
        inst.write('OUTPut1:STATe OFF')
        status = inst.query('OUTPut1:STATe?').strip()
        print(f'  ▶ LED OFF (STATe? = {status})')              # 0=OFF, 1=ON :contentReference[oaicite:16]{index=16}

        # OFF countdown
        for t in range(OFF_TIME, 0, -1):
            print(f'    {t:2d}s OFF…', end='\r')
            time.sleep(1)
        print(' ' * 40, end='\r')

def main():
    rm   = pyvisa.ResourceManager()
    inst = rm.open_resource(RESOURCE)
    inst.timeout = 5000  # ms

    # INITIAL SETUP
    inst.write('*RST')
    inst.write(f'SOURce1:CURRent:LIMIt:AMPLitude {MAX_LIMIT_MA/1000}')  # 0.2 A limit
    inst.write('OUTPut1:TERMinal 2')                               # select LED2 head :contentReference[oaicite:17]{index=17}
    print(f'Connected: {inst.query("*IDN?").strip()}')

    try:
        while True:
            countdown(INITIAL_COUNTDOWN)
            run_sequence(inst)
            # ask once after full sweep
            ans = input('\nRun another full pulse sequence? (y/n): ').strip().lower()
            if ans not in ('y','yes'):
                print('Exiting. Goodbye!')
                break

    except KeyboardInterrupt:
        print('\n⚠️  Interrupted! Turning LED off…')
    finally:
        inst.write('OUTPut1:STATe OFF')                            # safety shut-off :contentReference[oaicite:18]{index=18}

if __name__ == '__main__':
    main()


Connected: Thorlabs,DC2200,M00811426,1.4.0
                                        
Pulse 1/10 → 20 mA (10% of limit)
  ☑ [1 s] 20.1 mA | 2.70 V
  ▶ LED OFF (STATe? = 0)                
                                        
Pulse 2/10 → 40 mA (20% of limit)
  ☑ [1 s] 40.0 mA | 2.74 V
  ▶ LED OFF (STATe? = 0)                
                                        
Pulse 3/10 → 60 mA (30% of limit)
  ☑ [1 s] 60.1 mA | 2.78 V
  ▶ LED OFF (STATe? = 0)                
                                        
Pulse 4/10 → 80 mA (40% of limit)
  ☑ [1 s] 80.0 mA | 2.81 V
  ▶ LED OFF (STATe? = 0)                
                                        
Pulse 5/10 → 100 mA (50% of limit)
  ☑ [1 s] 100.1 mA | 2.84 V
  ▶ LED OFF (STATe? = 0)                
                                        
Pulse 6/10 → 120 mA (60% of limit)
  ☑ [1 s] 120.1 mA | 2.86 V
  ▶ LED OFF (STATe? = 0)                
                                        
Pulse 7/10 → 140 mA (70% of limit)
  ☑ [1 s] 140.1 mA 